# Random Restarts for the TSP Algorithm in Julia

Solves the Symmetric TSP using variations on the Random Restart approach to optimisation

# Add packages and imports

In [ ]:
#import Pkg
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [1]:
using Distributed # for parallel processes
using DataFrames
using CSV
using LinearAlgebra

In [3]:
#set up extra processes for parallel runs
println(nprocs())
println(nworkers())

1
1


In [4]:
addprocs(Sys.CPU_THREADS - 1);
println(nworkers())

15


In [7]:
@everywhere using Random

# Load data from csv

This is the 70 city problem from TSP lib.

In [8]:
st70 = CSV.read("data/st70.csv");

In [9]:
head(st70, 10)

,city,x,y
,Int64,Int64,Int64
1,1,64,96
2,2,80,39
3,3,69,23
4,4,72,42
5,5,48,67
6,6,58,43
7,7,81,34
8,8,79,17
9,9,30,23


# Functions

* euclidean_distance_matrix - compute cost matrix
* trim_cities - make problem instance smaller than 70 cities!
* tour_cost - compute cost of tour
* tweak! - randomly swap two cities in tour
* random_restarts - search by randomly shuffling tour for given time limit.
* hill_climb_with_random_restarts - shuffle tour then climb for a fraction of the total time limit; repeat until all time budget is used up.

In [10]:
"""
    euclidean_distance_matrix(cities)

Compute a matrix of euclidean distances between
city x, y coordinate pairs

# Arguments
- cities::Array: n x 2 matrix of x, y coordinates

# Examples
```julia-repl 
julia> using Random;

julia> Random.seed!(42);

julia> coords = rand(1:50, 5, 2)
5×2 Array{Int64,2}:
 16  42
 10  15
 44  36
 32  42
 28  39

julia> euclidean_distance_matrix(coords)
5×5 Array{Float64,2}:
  0.0     27.6586  28.6356  16.0     12.3693
 27.6586   0.0     39.9625  34.8281  30.0
 28.6356  39.9625   0.0     13.4164  16.2788
 16.0     34.8281  13.4164   0.0      5.0
 12.3693  30.0     16.2788   5.0      0.0
```
"""
function euclidean_distance_matrix(cities)
    nrows = size(cities)[1]
    matrix = zeros(nrows, nrows)
    
    row = 1
    
    for city1 in 1:nrows
        col = 1
        for city2 in 1:nrows
            matrix[row, col] = norm(cities[city1, 1:2]-cities[city2, 1:2])
            col+=1
        end
        row +=1
    end
        
    return matrix
end

euclidean_distance_matrix

In [11]:
trim_cities(df, ncities) = Array(df[2:end])[1:ncities, :]

trim_cities (generic function with 1 method)

In [12]:
@everywhere begin

"""
    tour_cost(tour, matrix)

Compute the travel cost of tour using
the cost matrix
"""
function tour_cost(tour, matrix)
    cost::Float64 = 0.0
    
    for i in 1:size(tour)[1] - 1
        cost += matrix[tour[i], tour[i+1]]
    end
    
    cost += matrix[tour[end], tour[1]]
    
    return cost
end

end

In [13]:
@everywhere begin
    
"""
    simple_tweak(tour)

Randomly select to elements within the 
vector tour and swap them
"""
function simple_tweak(tour)
    sample = rand(1:size(tour)[1], 1, 2)
    tour[sample[1]], tour[sample[2]] = tour[sample[2]], tour[sample[1]]
    return tour
end

end

In [14]:
function tweak_two_opt(tour)
    sample = rand(1:size(tour)[1], 1, 2)
    tour = reverse(tour, tour[sample[1]], tour[sample[2]])
    return tour
end

tweak_two_opt (generic function with 1 method)

# Algorithm

In [15]:
@everywhere function random_restarts(init_solution, matrix; time_limit=2)
    best = copy(init_solution)
    best_cost = -tour_cost(best, matrix)
    
    start = time()
    iter = 0
    while (time() - start) < time_limit
        iter += 1
        neighbour = shuffle(copy(best))
        neighbour_cost = -tour_cost(neighbour, matrix)
        
        if neighbour_cost > best_cost
            best, best_cost = neighbour, neighbour_cost
        end
    end
    
    return -best_cost, best, iter
end 

In [16]:
@everywhere function random_restarts(init_solution, matrix; maxiter=1000)
    best = copy(init_solution)
    neighbour = copy(init_solution)
    best_cost = -tour_cost(best, matrix)
        
    iter = 1
    while iter < maxiter
        iter += 1
        neighbour = shuffle(neighbour)
        neighbour_cost = -tour_cost(neighbour, matrix)
        
        if neighbour_cost > best_cost
            best, best_cost = neighbour, neighbour_cost
        end
    end
    
    return -best_cost, best
end 

In [17]:
@everywhere begin
"""
    simple_tweak(tour)

Randomly select to elements within the 
vector tour and swap them
"""
function simple_tweak(tour)
    sample = rand(1:size(tour)[1], 1, 2)
    tour[sample[1]], tour[sample[2]] = tour[sample[2]], tour[sample[1]]
    return tour
end

end

In [18]:
@everywhere begin
"""
    tweak_two_opt(tour)

Randomly select to elements within the 
vector tour and reverse the route

e.g. 
[1, 2, 3, 4, 5, 6, 7, 8]

if elements 2 and 5 are selected then the result is:
[1, 5, 4, 3, 2, 6, 7, 8]
"""
function tweak_two_opt(tour)
    sample = rand(1:size(tour)[1], 1, 2)
    tour = reverse(tour, tour[sample[1]], tour[sample[2]])
    return tour
end

end

In [19]:
"""
    hill_climb_with_random_restarts(init_solution, matrix; time_limit=2)

Breaks runtime into hill climbing for a period of time and then restarting
the climb from a different random initial solution.

"""

"    hill_climb_with_random_restarts(init_solution, matrix; time_limit=2)\n\nBreaks runtime into hill climbing for a period of time and then restarting\nthe climb from a different random initial solution.\n\n"

In [20]:
@everywhere function hill_climb_with_random_restarts(init_solution, matrix; time_limit=2.0, 
                                         tweak=simple_tweak)

    S = shuffle(init_solution)
    S_cost = -tour_cost(S, matrix)
    
    best, best_cost = copy(S), S_cost
    
    start = time()
    while (time() - start) < time_limit
        climbing_time = rand(0: time_limit - (time() - start))
        climb_start = time()
        
        while (time() - climb_start) < climbing_time
            R = tweak(copy(S))
            R_cost = -tour_cost(R, matrix)

            if R_cost > S_cost
                S, S_cost = copy(R), R_cost
            end
        
        end
        
        if S_cost > best_cost
            best, best_cost = copy(S), S_cost
        end
        
        S = shuffle(init_solution)
        S_cost -tour_cost(S, matrix)
    end
    
    return -best_cost, best
    
end

# Example solution

In [21]:
Random.seed!(42);

n_patients = 20
coords = trim_cities(st70, n_patients);
matrix = euclidean_distance_matrix(coords);
tour = [i for i in 1:n_patients];

# tweak with 2 opt.
@time result1 = hill_climb_with_random_restarts(tour, matrix, time_limit=2.0,
                                                tweak=tweak_two_opt)

#reset random seed
Random.seed!(42);

#extra time
@time result2 = hill_climb_with_random_restarts(tour, matrix, time_limit=5.0,
                                                tweak=tweak_two_opt)

println(result1)
println(result2)

  2.144752 seconds (15.87 M allocations: 2.858 GiB, 7.94% gc time)
  5.000005 seconds (55.25 M allocations: 9.965 GiB, 11.97% gc time)
(396.7043800775278, [6, 5, 10, 11, 12, 17, 9, 20, 14, 3, 8, 7, 2, 19, 15, 13, 1, 16, 4, 18])
(362.3478303531909, [14, 20, 8, 19, 15, 13, 1, 16, 5, 10, 11, 12, 9, 17, 6, 18, 4, 2, 7, 3])


In [ ]:
#weird behaviour on the parrallel run so use maxiter instead.
@time random_restarts(tour, matrix, maxiter=5_000_000)

In [ ]:
@time hill_climb_with_random_restarts(tour, matrix)

# Running the algorithms multiple times in parrallel

Note the use of @everywhere in the function definitions. This means that different workers can access them.

In [40]:
# schedule the parallel jobs

n_jobs = Sys.CPU_THREADS - 1
jobs = []

for i in 1:n_jobs
    push!(jobs, remotecall(random_restarts, i+1, tour, matrix, maxiter=5_000_000))
end

In [41]:
jobs

15-element Array{Any,1}:
 Future(2, 1, 380, nothing)
 Future(3, 1, 381, nothing)
 Future(4, 1, 382, nothing)
 Future(5, 1, 383, nothing)
 Future(6, 1, 384, nothing)
 Future(7, 1, 385, nothing)
 Future(8, 1, 386, nothing)
 Future(9, 1, 387, nothing)
 Future(10, 1, 388, nothing)
 Future(11, 1, 389, nothing)
 Future(12, 1, 390, nothing)
 Future(13, 1, 391, nothing)
 Future(14, 1, 392, nothing)
 Future(15, 1, 393, nothing)
 Future(16, 1, 394, nothing)

In [42]:
#this is currently doing something odd when running with time!  so restrict to maxiter
@time results = [fetch(job) for job in jobs]

  1.009576 seconds (1.42 k allocations: 53.031 KiB)


15-element Array{Tuple{Float64,Array{Int64,1}},1}:
 (507.6272724833943, [15, 2, 4, 6, 5, 10, 9, 12, 17, 11, 1, 13, 16, 18, 19, 8, 20, 14, 3, 7])
 (510.22548190280025, [17, 10, 12, 5, 11, 1, 16, 13, 15, 19, 2, 4, 3, 20, 8, 7, 14, 6, 18, 9])
 (529.7590382968284, [9, 6, 4, 18, 15, 13, 1, 16, 10, 11, 5, 7, 2, 20, 8, 19, 14, 3, 12, 17])
 (515.7013623203869, [20, 14, 9, 12, 5, 11, 16, 1, 13, 10, 17, 6, 3, 18, 7, 19, 8, 4, 15, 2])
 (516.899529331729, [11, 10, 9, 17, 12, 3, 8, 19, 2, 7, 4, 20, 14, 18, 15, 13, 1, 16, 5, 6])
 (498.1969900233141, [10, 5, 1, 13, 16, 11, 12, 9, 17, 8, 18, 6, 2, 7, 19, 20, 14, 3, 15, 4])
 (517.9637773845324, [14, 20, 2, 18, 10, 11, 5, 16, 1, 13, 12, 6, 17, 9, 4, 7, 15, 19, 3, 8])
 (500.8471103351681, [4, 2, 7, 19, 15, 18, 9, 17, 20, 14, 6, 5, 11, 12, 10, 16, 1, 13, 3, 8])
 (485.87152480349073, [6, 11, 12, 20, 14, 3, 8, 2, 19, 7, 4, 18, 15, 1, 13, 16, 10, 5, 17, 9])
 (484.58061695450897, [11, 5, 16, 1, 13, 2, 19, 15, 18, 7, 3, 20, 14, 8, 4, 9, 17, 6, 12, 10])
 (524.1

In [23]:
job = remotecall(random_restarts, 2, tour, matrix, maxiter=5_000_000)
fetch(job)

Future(2, 1, 258, nothing)

(519.0610284426596, [6, 12, 17, 9, 2, 4, 7, 15, 19, 8, 20, 14, 3, 5, 11, 13, 1, 16, 10, 18])